In [1]:
import d3rlpy

# get CartPole dataset
dataset, env = d3rlpy.datasets.get_cartpole()

2024-02-06 21:44.57 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(4,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-02-06 21:44.57 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2024-02-06 21:44.57 [info     ] Action size has been automatically determined. action_size=2


In [ ]:
len(dataset.episodes)

In [2]:
episodes = dataset.episodes

In [ ]:
ep_lengths = []
for ep in episodes:
    ep_lengths.append(ep.observations.shape[0])

In [ ]:
import numpy as np
np.min(ep_lengths), np.max(ep_lengths), np.mean(ep_lengths)

In [ ]:
np.random.randint(100)

In [34]:
import h5py
f = "../offline-rl-experiments/d3rlpy_data/cartpole_replay_v1.1.0.h5"
with h5py.File(f, "r") as h5:
    observations = h5["observations"][()]
    actions = h5["actions"][()]
    rewards = h5["rewards"][()]
    terminals = h5["terminals"][()]

if "episode_terminals" in h5:
    episode_terminals = h5["episode_terminals"][()]
else:
    episode_terminals = None

In [35]:
episode_terminals

In [23]:
observations.shape

(99866, 4)

In [46]:
# load dataset
from d3rlpy.dataset import load_v1
with open("../offline-rl-experiments/d3rlpy_data/cartpole_replay_v1.1.0.h5", "rb") as f:
    episodes = load_v1(f)

In [47]:
episodes[0]['observations']

TypeError: 'Episode' object is not subscriptable

In [26]:
data_path = "/home/ssk/Desktop/master-thesis/master-thesis-personalization/data/dt-datasets/movielens/train-test-sets/mlens-train-trajectories-movies-as-actions-reduced-from-768-to-32.pkl"

import pickle
with open(data_path, "rb") as f:
    episodes_mlens = pickle.load(f)

# with open(data_path, "r") as f:
#     observations = h5["observations"][()]
#     actions = h5["actions"][()]
#     rewards = h5["rewards"][()]
#     terminals = h5["terminals"][()]

In [50]:
import numpy as np
observations_mlens = []
observations_mlens = np.concatenate([ep['observations'] for ep in episodes_mlens])
actions_mlens = np.concatenate([ep['actions'] for ep in episodes_mlens])
rewards_mlens = np.concatenate([ep['rewards'] for ep in episodes_mlens])
terminals_mlens = np.concatenate([ep['terminals'] for ep in episodes_mlens])

timeouts = None

In [51]:
from d3rlpy.dataset import EpisodeGenerator

episode_generator = EpisodeGenerator(
    observations=observations_mlens,
    actions=actions_mlens,
    rewards=rewards_mlens,
    terminals=terminals_mlens,
    timeouts=timeouts,
)

episodes_generated_mlens = episode_generator()

In [52]:
from d3rlpy.dataset import ReplayBuffer, InfiniteBuffer

dataset = ReplayBuffer(
    InfiniteBuffer(),
    episodes=episodes_generated_mlens,
    transition_picker=None,
    trajectory_slicer=None,
)

2024-02-06 22:13.20 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(32,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(32,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2024-02-06 22:13.20 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-02-06 22:13.20 [info     ] Action size has been automatically determined. action_size=32


In [53]:
dataset.episodes[0].observations.shape

(227, 32)

In [54]:
cql = d3rlpy.algos.CQLConfig().create(device='cuda:0')

In [59]:
# start training
cql.fit(
    dataset,
    n_steps=1000,
    n_steps_per_epoch=1000,
    # evaluators={
    #     'environment': d3rlpy.metrics.EnvironmentEvaluator(env), # evaluate with CartPole-v1 environment
    # },
)

2024-02-06 22:18.06 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(32,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(32,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=32)
2024-02-06 22:18.06 [info     ] Directory is created at d3rlpy_logs/CQL_20240206221806
2024-02-06 22:18.06 [warning  ] Skip building models since they're already built.
2024-02-06 22:18.06 [info     ] Parameters                     params={'observation_shape': [32], 'action_size': 32, 'config': {'type': 'cql', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'actor_learning_rate': 0.0001, 'critic_learning_rate': 0.0003, 'temp_learning_rate': 0.0001, 'alpha_learning_rate': 0.0001, 'actor_opt

Epoch 1/1:   0%|          | 0/1000 [00:00<?, ?it/s]

2024-02-06 22:18.35 [info     ] CQL_20240206221806: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.010275593996047973, 'time_algorithm_update': 0.018204174041748046, 'critic_loss': 150.56041670227052, 'conservative_loss': 145.3034454650879, 'alpha': 1.310522206068039, 'actor_loss': -32.764680757522584, 'temp': 0.7846612847447395, 'temp_loss': 42.24916088104248, 'time_step': 0.02856256651878357} step=1000
2024-02-06 22:18.35 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20240206221806/model_1000.d3


[(1,
  {'time_sample_batch': 0.010275593996047973,
   'time_algorithm_update': 0.018204174041748046,
   'critic_loss': 150.56041670227052,
   'conservative_loss': 145.3034454650879,
   'alpha': 1.310522206068039,
   'actor_loss': -32.764680757522584,
   'temp': 0.7846612847447395,
   'temp_loss': 42.24916088104248,
   'time_step': 0.02856256651878357})]

In [ ]:
# Create a random dataset for testing
total_episodes = 100
for i in range(total_episodes):
    ep_length = np.random.randint(100)
    
    for j in ep_length:


In [ ]:
# setup CQL algorithm
cql = d3rlpy.algos.DiscreteCQLConfig().create(device='cuda')

In [ ]:
# start training
cql.fit(
    dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={
        'environment': d3rlpy.metrics.EnvironmentEvaluator(env), # evaluate with CartPole-v1 environment
    },
)

In [ ]:
import gym
env = gym.make("CartPole-v1", render_mode="rgb_array")
d3rlpy.metrics.evaluate_qlearning_with_environment(cql, env)

#### Continuous CQL

In [ ]:
import d3rlpy

# prepare dataset
dataset, env = d3rlpy.datasets.get_d4rl('hopper-medium-v0')

# # prepare algorithm
# cql = d3rlpy.algos.CQLConfig().create(device='cuda:0')

# # train
# cql.fit(
#     dataset,
#     n_steps=100000,
#     evaluators={"environment": d3rlpy.metrics.EnvironmentEvaluator(env)},
# )

In [ ]:
env.step(0.45)

In [ ]:
dataset.episodes[0].observations.shape

In [ ]:
import d4rl

In [ ]:
a = [1, 2, 3]


In [ ]:
a[-2:]